# ყველა ფაილის მისამართის ამოღება

In [67]:
!pip install librosa
!pip install pydub

In [1]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [2]:
def convert_to_wav(files):
    for src in files:
        dst = src[:-3] + "wav"
        s = AudioSegment.from_file(src)
        s.export(dst, format="wav")

In [4]:
folders = os.listdir('data')
for sub_dir in folders:
#     print(sub_dir)
    mp3_files = glob.glob(os.getcwd() + '/data/' + sub_dir + '/*.mp3')
    convert_to_wav(mp3_files)
#     m4a_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.m4a')
#     mp4_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.mp4')
#     aac_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.aac')
#     files = mp3_files + m4a_files + mp4_files + aac_files
#     რატომღაც ამას არ შვება :(
#     convert_to_wav(files) 
#     print(mp3_files)

In [5]:
import pandas as pd
import librosa
import librosa.display

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [12]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
        zero_crossing_rate = get_zero_crossing_rate(audio)
#         other features
        return [mfccs, zero_crossing_rate]
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 

In [13]:
def get_all_features():
    folders = os.listdir('data')
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/data/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'zero_crossing_rate' , 'class_label'])
    return data

In [14]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccsscaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, 61), mfccsscaled - (40,)
    return mfccsscaled

In [15]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
#     rate - (1, 61)
    return rate

In [16]:
def get_spectral_centroid(audio):
    #SPECTRAL_CENTROID
    sp = librosa.feature.spectral_centroid(audio)
#     sp - (1, 61)
    return sp

In [17]:
def get_spectral_roll_ff(audio, sr):
    # Spectral-roll off
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
#     a - (1, 61)
    return a

In [18]:
def get_spectral_flux(audio, sr):
    # Spectral flux
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
#     onset_env - (61,)
    return onset_env

In [19]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, 61)
    return [chroma_stft, chroma_cens, chroma_cq]

In [20]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, 61)
    return pitches

data = get_all_features()

In [280]:
data = get_all_features()

/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')
/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [353]:
data

,mfccs,zero_crossing_rate,class_label
0,"[-485.77536, 103.56779, 5.1006136, 12.516183, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
1,"[-293.36374, 99.50089, 54.90203, 52.981934, 21...","[[0.00244140625, 0.0048828125, 0.00732421875, ...",1
2,"[-491.02005, 118.41104, 11.42216, 14.3336115, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",1
3,"[-440.54318, 48.62018, -13.115458, 39.109562, ...","[[0.19384765625, 0.28369140625, 0.39306640625,...",1
4,"[-369.51532, 71.60074, 18.153286, 36.65747, 5....","[[0.02294921875, 0.05078125, 0.0634765625, 0.0...",1
...,...,...,...
140,"[-333.4615, 78.47683, -11.2027025, 34.041134, ...","[[0.0, 0.01953125, 0.10400390625, 0.138671875,...",5
141,"[-361.54678, 111.94943, 69.74507, 25.497889, 1...","[[0.1123046875, 0.11767578125, 0.1279296875, 0...",5
142,"[-260.16925, 111.11569, 54.97757, 40.923706, 1...","[[0.12548828125, 0.130859375, 0.1357421875, 0....",5
143,"[-257.8959, 84.435715, -3.2791898, 42.858597, ...","[[0.0, 0.0595703125, 0.10205078125, 0.13330078...",5


In [282]:
labels = [[i] for i in data['class_label']]
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(labels)
print(y_onehot.shape)
y =np.array(labels)

(145, 5)


/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [336]:
X = data.drop(columns=['class_label', 'zero_crossing_rate'])
X = X.values
X[0][0].shape
# X
# type(X[0][0])
ls = [] #featurebi gvinda da magitom
for i in range(X.shape[0]):
    ls.append(X[i][0])
ls = np.array(ls)
ls.shape

(145, 40)

 # ნეირონული ქსელის კოდი

In [256]:
def sigmoid(z):
#     print(z.shape) 
    return 1 / (1 + np.exp(-z))

In [338]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [269]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [343]:
# initial setup
input_size = ls.shape[1]
hidden_size = 25
num_labels = y_onehot.shape[1]
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls.shape[0]
ls = np.matrix(ls)
y = np.matrix(y_onehot)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((25, 41), (5, 26))

In [344]:
a1, z2, a2, z3, h = forward_propagate(ls, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((145, 41), (145, 25), (145, 26), (145, 5), (145, 5))

In [345]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [346]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [348]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls, y_onehot, learning_rate)
J, grad.shape


(3.5102641414287357, (1155,))

In [351]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls, y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 1.833400293399364
     jac: array([-0.00128446,  0.59556136, -0.10624242, ...,  0.01653273,
        0.01965055, -0.00125899])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 29
  status: 3
 success: False
       x: array([-0.08518204, -0.02837108, -0.04215426, ..., -0.27492505,
        0.10382888, -0.18255394])

In [352]:
ls = np.matrix(ls)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

array([[1],
       [2],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [4],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [4],
       [2],
       [2],
       [5],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [4],
       [4],
       [2],
       [2],
       [2],
       [4],
       [2],
       [4],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [4],
       [4],
       [3],
       [5],
       [3],
       [3],
       [4],
       [3],
       [2],
       [2],
       [4],
       [5],
       [2],
       [3],
       [3],
       [4],
       [4],
       [5],
       [4],
       [2],
       [5],
       [4],
       [4],
       [4],
       [3],
    